<a href="https://colab.research.google.com/github/ankesh86/RecommendationSystems/blob/main/HybridRecommendationSystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip -q install scipy lightfm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [43]:
#libraries
import pandas as pd
import numpy as np
from scipy.sparse import coo_matrix
from lightfm import LightFM
from lightfm.evaluation import auc_score
import time
import sklearn
from sklearn import model_selection

In [9]:
import pandas as pd

# URL of the Excel file (raw URL)
url = 'https://raw.githubusercontent.com/Apress/applied-recommender-systems-python/master/Data/Rec_sys_data.xlsx'

# Read the data into a DataFrame
order_df = pd.read_excel(url,'order')
#customer_df
customer_df = pd.read_excel(url,'customer')
#products data
product_df = pd.read_excel(url,'product')

# Display the first few rows of the DataFrame
print(order_df.head())
print(customer_df.head())
print(product_df.head())

   InvoiceNo StockCode  Quantity         InvoiceDate        DeliveryDate  \
0     536365    84029E         6 2010-12-01 08:26:00 2010-12-02 08:26:00   
1     536365     71053         6 2010-12-01 08:26:00 2010-12-02 08:26:00   
2     536365     21730         6 2010-12-01 08:26:00 2010-12-03 08:26:00   
3     536365    84406B         8 2010-12-01 08:26:00 2010-12-03 08:26:00   
4     536365     22752         2 2010-12-01 08:26:00 2010-12-04 08:26:00   

   Discount%        ShipMode  ShippingCost  CustomerID  
0       0.20      ExpressAir         30.12       17850  
1       0.21      ExpressAir         30.12       17850  
2       0.56     Regular Air         15.22       17850  
3       0.30     Regular Air         15.22       17850  
4       0.57  Delivery Truck          5.81       17850  
   CustomerID  Gender  Age  Income  Zipcode Customer Segment
0       13089    male   53    High     8625   Small Business
1       15810  female   22     Low    87797   Small Business
2       15556  fem

In [10]:
#merging all three dataframes
merged_df = pd.merge(order_df, customer_df, left_on=['CustomerID'], right_on=['CustomerID'],how='left')
merged_df = pd.merge(merged_df, product_df, left_on=['StockCode'], right_on=['StockCode'],how='left')
merged_df.head()

,InvoiceNo,StockCode,Quantity,InvoiceDate,DeliveryDate,Discount%,ShipMode,ShippingCost,CustomerID,Gender,Age,Income,Zipcode,Customer Segment,Product Name,Description,Category,Brand,Unit Price
0,536365,84029E,6,2010-12-01 08:26:00,2010-12-02 08:26:00,0.20,ExpressAir,30.12,17850,female,48,Medium,84306,Middle class,"3 1/2""W x 20""D x 20""H Funston Craftsman Smooth...",Our Rustic Collection is an instant classic. O...,Home Improvement|Hardware|Brackets and Angle I...,Ekena Milwork,199.11
1,536365,71053,6,2010-12-01 08:26:00,2010-12-02 08:26:00,0.21,ExpressAir,30.12,17850,female,48,Medium,84306,Middle class,Awkward Styles Shamrock Flag St. Patrick's Day...,Our St Patrick's Day Collection is perfect for...,Clothing|Men|Mens T-Shirts & Tank Tops|Mens Gr...,Awkward Styles,23.95
2,536365,21730,6,2010-12-01 08:26:00,2010-12-03 08:26:00,0.56,Regular Air,15.22,17850,female,48,Medium,84306,Middle class,Ebe Men Black Rectangle Half Rim Spring Hinge ...,Count on EBE for all of your eye correction ne...,Health|Home Health Care|Daily Living Aids,Eye Buy Express,26.99
3,536365,84406B,8,2010-12-01 08:26:00,2010-12-03 08:26:00,0.30,Regular Air,15.22,17850,female,48,Medium,84306,Middle class,MightySkins Skin Decal Wrap Compatible with Ap...,Mightyskins are removable vinyl skins for prot...,Electronics|Electronics Learning Center|Ads Fr...,Mightyskins,14.99
4,536365,22752,2,2010-12-01 08:26:00,2010-12-04 08:26:00,0.57,Delivery Truck,5.81,17850,female,48,Medium,84306,Middle class,awesome since 1948 - 69th birthday gift t-shir...,awesome since 1948 - 69th birthday gift t-shir...,Clothing|Men|Mens T-Shirts & Tank Tops|Mens T-...,Shirtinvaders,49.33


# Data Preperation
- user-to-product interaction matrix
- product-to-features interaction mappings

In [11]:
def unique_users(data, column):
  return np.sort(data[column].unique())

def unique_items(data, column):
  item_list = data[column].unique()
  return item_list

user_list = unique_users(order_df, 'CustomerID')
item_list = unique_items(product_df,'Product Name')


In [12]:
user_list

array([12346, 12347, 12348, ..., 18282, 18283, 18287])

In [14]:
def features_to_add(customer, column1, column2, column3):
  customer1 = customer[column1]
  customer2 = customer[column2]
  customer3 = customer[column3]
  return pd.concat([customer1, customer3, customer2], ignore_index=True).unique()

feature_unique_list = features_to_add(customer_df, 'Customer Segment','Age', 'Gender')
feature_unique_list

array(['Small Business', 'Corporate', 'Middle class', 'male', 'female',
       53, 22, 29, 36, 48, 45, 47, 23, 39, 34, 52, 51, 35, 19, 26, 37, 18,
       20, 21, 41, 31, 28, 50, 38, 30, 25, 32, 55, 43, 54, 49, 40, 33, 44,
       46, 42, 27, 24], dtype=object)

In [35]:
def mapping(user_list, item_list, feature_unique_list):
  #creating empty output dicts
  user_to_index_mapping = {}
  index_to_user_mapping = {}

  #Create id mappings to convert user_id
  for user_index, user_id in enumerate(user_list):
    user_to_index_mapping[user_id] = user_index
    index_to_user_mapping[user_index] = user_id

  item_to_index_mapping = {}
  index_to_item_mapping = {}
  #create id mappings to convert item_id
  for item_index, item_id in enumerate(item_list):
    item_to_index_mapping[item_id] = item_index
    index_to_item_mapping[item_index] = item_id


  feature_to_index_mapping = {}
  index_to_feature_mapping = {}
  #create ID mappings to convert feature_id
  for feature_index, feature_id in enumerate(feature_unique_list):
    feature_to_index_mapping[feature_id] = feature_index
    index_to_feature_mapping[feature_index] = feature_id

  return user_to_index_mapping, index_to_user_mapping, item_to_index_mapping, index_to_item_mapping, feature_to_index_mapping, index_to_feature_mapping


# generate mapping, LightFM library can't read other than (integer) index
user_to_index_mapping, index_to_user_mapping, \
           item_to_index_mapping, index_to_item_mapping, \
           feature_to_index_mapping, index_to_feature_mapping = mapping(user_list, item_list, feature_unique_list)


In [36]:
user_to_index_mapping, index_to_user_mapping, item_to_index_mapping, index_to_item_mapping, feature_to_index_mapping, index_to_feature_mapping = mapping(user_list, item_list, feature_unique_list)

In [17]:
#user-to-product relationship and calculating total quantity per user
user_to_product = merged_df[['CustomerID','Product Name','Quantity']]
#calc total qty. (sum) per customer-product
user_to_product = user_to_product.groupby(['CustomerID', 'Product Name']).agg({'Quantity':'sum'}).reset_index()
user_to_product.tail()

,CustomerID,Product Name,Quantity
138397,18287,Sport-Tek Ladies PosiCharge Competitor Tee,24
138398,18287,Ultra Sleek And Spacious Pearl White Lacquer 1...,6
138399,18287,"Union 3"" Female Ports Stainless Steel Pipe Fit...",12
138400,18287,awesome since 1948 - 69th birthday gift t-shir...,4
138401,18287,billyboards Porcelain Menu Chalkboard,6


In [19]:
#product-to-features relationship
product_to_feature = merged_df[['Product Name','Customer Segment','Quantity']]

#calc total qty. per customer per segment
product_to_feature = product_to_feature.groupby(['Product Name','Customer Segment']).agg({'Quantity':'sum'}).reset_index()
product_to_feature.head()

,Product Name,Customer Segment,Quantity
0,"""In Vinyl W.e Trust"" Rasta Quote Men's T-shirt",Corporate,712
1,"""In Vinyl W.e Trust"" Rasta Quote Men's T-shirt",Middle class,272
2,"""In Vinyl W.e Trust"" Rasta Quote Men's T-shirt",Small Business,388
3,"""Soccer"" Vinyl Graphic - Large - Ivory",Corporate,1940
4,"""Soccer"" Vinyl Graphic - Large - Ivory",Middle class,1418


In [20]:
# split the user-to-product relationship into test - train
user_to_product_train,user_to_product_test = model_selection.train_test_split(user_to_product, test_size=0.33, random_state=42)

print(user_to_product_train.shape)
print(user_to_product_test.shape)



(92729, 3)
(45673, 3)


In [28]:


def interactions(data, row, col, value, row_map, col_map):

    #converting the row with its given mappings
    row = data[row].apply(lambda x: row_map[x]).values
    #converting the col with its given mappings
    col = data[col].apply(lambda x: col_map[x]).values
    value = data[value].values
    #returning the interaction matrix
    return coo_matrix((value, (row, col)), shape = (len(row_map), len(col_map)))

In [37]:
# generate user_item_interaction_matrix for train data
user_to_product_interaction_train = interactions(user_to_product_train, "CustomerID",
                                                    "Product Name", "Quantity", user_to_index_mapping, item_to_index_mapping)


In [38]:
#for train
user_to_product_interaction_train = interactions(user_to_product_train, 'CustomerID', 'Product Name', 'Quantity', user_to_index_mapping, item_to_index_mapping)

#for test
user_to_product_interaction_test = interactions(user_to_product_test, 'CustomerID', 'Product Name', 'Quantity', user_to_index_mapping, item_to_index_mapping)

print(user_to_product_interaction_train)

  (2124, 230)	10
  (1060, 268)	16
  (1633, 295)	3
  (113, 45)	20
  (3100, 44)	54
  (3110, 25)	4
  (2133, 126)	6
  (525, 176)	2
  (2711, 45)	26
  (281, 487)	25
  (1378, 349)	27
  (2356, 18)	12
  (3248, 6)	24
  (1295, 339)	1
  (1229, 4)	120
  (3153, 36)	24
  (2441, 54)	1
  (1522, 124)	4
  (1436, 15)	8
  (3613, 137)	12
  (265, 324)	1
  (3309, 220)	24
  (2943, 3)	18
  (2021, 407)	5
  (2516, 187)	18
  :	:
  (64, 8)	24
  (3406, 109)	1
  (3219, 12)	12
  (2196, 96)	25
  (3409, 236)	12
  (2216, 172)	1
  (2720, 60)	5
  (121, 326)	6
  (1670, 63)	31
  (1555, 4)	6
  (22, 112)	12
  (1670, 8)	15
  (1736, 41)	10
  (1101, 256)	8
  (415, 152)	12
  (3303, 154)	2
  (2955, 8)	16
  (2276, 27)	1
  (3612, 527)	12
  (1446, 299)	192
  (2888, 238)	60
  (3140, 35)	2
  (2727, 591)	7
  (3451, 194)	1
  (3203, 15)	4


In [40]:
product_to_feature_interaction = interactions(product_to_feature, "Product Name", "Customer Segment","Quantity", item_to_index_mapping, feature_to_index_mapping)

# **Model**

In [42]:
#intializing model with wrap loss function
model_with_features = LightFM(loss="warp")
start = time.time()

model_with_features.fit_partial(user_to_product_interaction_train,
                                user_features=None,
                                item_features=product_to_feature_interaction,
                                sample_weight=None,
                                epochs=1,
                                num_threads = 4,
                                verbose=False)

end = time.time()
print("time taken = {0:.{1}f} sec".format(end - start,2))

time taken = 0.22 sec


In [44]:
start = time.time()
#calcualting area under curve(AUC) score for the validation
auc_with_features = auc_score(model=model_with_features,
                              test_interactions = user_to_product_interaction_test,
                              train_interactions = user_to_product_interaction_train,
                              item_features = product_to_feature_interaction,
                              num_threads=4,
                              check_intersections=False
                              )
end = time.time()

print("time taken = {0:.{1}f} sec".format(end - start,2))
print("Average AUC without adding item-feature interaction = {0:.{1}f}".format(auc_with_features.mean(),2))

time taken = 0.29 sec
Average AUC without adding item-feature interaction = 0.15


In [45]:
#intializing model with wrap loss function
model_with_features = LightFM(loss="logistic")
start = time.time()

model_with_features.fit_partial(user_to_product_interaction_train,
                                user_features=None,
                                item_features=product_to_feature_interaction,
                                sample_weight=None,
                                epochs=1,
                                num_threads = 4,
                                verbose=False)

end = time.time()
print("time taken = {0:.{1}f} sec".format(end - start,2))

start = time.time()
#calcualting area under curve(AUC) score for the validation
auc_with_features = auc_score(model=model_with_features,
                              test_interactions = user_to_product_interaction_test,
                              train_interactions = user_to_product_interaction_train,
                              item_features = product_to_feature_interaction,
                              num_threads=4,
                              check_intersections=False
                              )
end = time.time()

print("time taken = {0:.{1}f} sec".format(end - start,2))
print("Average AUC without adding item-feature interaction = {0:.{1}f}".format(auc_with_features.mean(),2))

time taken = 0.07 sec
time taken = 0.28 sec
Average AUC without adding item-feature interaction = 0.89


In [46]:
#intializing model with wrap loss function
model_with_features = LightFM(loss="bpr")
start = time.time()

model_with_features.fit_partial(user_to_product_interaction_train,
                                user_features=None,
                                item_features=product_to_feature_interaction,
                                sample_weight=None,
                                epochs=1,
                                num_threads = 4,
                                verbose=False)

end = time.time()
print("time taken = {0:.{1}f} sec".format(end - start,2))

start = time.time()
#calcualting area under curve(AUC) score for the validation
auc_with_features = auc_score(model=model_with_features,
                              test_interactions = user_to_product_interaction_test,
                              train_interactions = user_to_product_interaction_train,
                              item_features = product_to_feature_interaction,
                              num_threads=4,
                              check_intersections=False
                              )
end = time.time()

print("time taken = {0:.{1}f} sec".format(end - start,2))
print("Average AUC without adding item-feature interaction = {0:.{1}f}".format(auc_with_features.mean(),2))

time taken = 0.22 sec
time taken = 0.44 sec
Average AUC without adding item-feature interaction = 0.49


In [47]:
#intializing model with wrap loss function
model_with_features = LightFM(loss="logistic")
start = time.time()

model_with_features.fit_partial(user_to_product_interaction_train,
                                user_features=None,
                                item_features=product_to_feature_interaction,
                                sample_weight=None,
                                epochs=10,
                                num_threads = 20,
                                verbose=False)

end = time.time()
print("time taken = {0:.{1}f} sec".format(end - start,2))

start = time.time()
#calcualting area under curve(AUC) score for the validation
auc_with_features = auc_score(model=model_with_features,
                              test_interactions = user_to_product_interaction_test,
                              train_interactions = user_to_product_interaction_train,
                              item_features = product_to_feature_interaction,
                              num_threads=4,
                              check_intersections=False
                              )
end = time.time()

print("time taken = {0:.{1}f} sec".format(end - start,2))
print("Average AUC without adding item-feature interaction = {0:.{1}f}".format(auc_with_features.mean(),2))

time taken = 0.83 sec
time taken = 0.27 sec
Average AUC without adding item-feature interaction = 0.89


In [48]:
#merging train and test data
def train_test_merge(training_data, testing_data):
  train_dict = {}

  for row, col, data in zip(testing_data.row, testing_data.col, testing_data.data):
    train_dict[(row, col)] = max(data, train_dict.get((row, col), 0))

  #converting to the row
  row_list = []
  col_list = []
  data_list = []
  for row, col in train_dict:
    row_list.append(row)
    col_list.append(col)
    data_list.append(train_dict[(row, col)])

  # converting to np array
  row_list = np.array(row_list)
  col_list = np.array(col_list)
  data_list = np.array(data_list)

  return coo_matrix((data_list, (row_list, col_list)), shape = (training_data.shape[0], training_data.shape[1]))



In [49]:
user_to_product_interaction = train_test_merge(user_to_product_interaction_train, user_to_product_interaction_test)

# **Final Model**

In [54]:
# retraining the final model with combined dataset

final_model = LightFM(loss = "logistic",no_components=30)

# fitting to combined dataset

start = time.time()
#===================

final_model.fit(user_to_product_interaction,
          user_features=None,
          item_features=product_to_feature_interaction,
          sample_weight=None,
          epochs=10,
          num_threads=20,
          verbose=False)

#===================
end = time.time()
print("time taken = {0:.{1}f} seconds".format(end - start, 2))

time taken = 0.91 seconds


In [55]:
def get_recommendations(model,user,items,user_to_product_interaction_matrix,user2index_map,product_to_feature_interaction_matrix):

# getting the userindex

    userindex = user2index_map.get(user, None)

    if userindex == None:
        return None

    users = userindex

    # products already bought

    known_positives = items[user_to_product_interaction_matrix.tocsr()[userindex].indices]
    print('User index =',users)

    # scores from model prediction
    scores = model.predict(user_ids = users, item_ids = np.arange(user_to_product_interaction_matrix.shape[1]),item_features=product_to_feature_interaction_matrix)

    # top items

    top_items = items[np.argsort(-scores)]

    # printing out the result
    print("User %s" % user)
    print("     Known positives:")

    for x in known_positives[:10]:
        print("                  %s" % x)


    print("     Recommended:")

    for x in top_items[:10]:
        print("                  %s" % x)

In [56]:
get_recommendations(final_model,17017,item_list,user_to_product_interaction,user_to_index_mapping,product_to_feature_interaction)

User index = 2888
User 17017
     Known positives:
                  Mediven Sheer and Soft 15-20 mmHg Thigh w/ Lace Silicone Top Band CT Wheat II - Ankle 8-8.75 inches
                  MightySkins Skin Decal Wrap Compatible with OtterBox Sticker Protective Cover 100's of Color Options
                  MightySkins Skin Decal Wrap Compatible with DJI Sticker Protective Cover 100's of Color Options
                  Ebe Reading Glasses Mens Womens Tortoise Bold Rectangular Full Frame Anti Glare grade ckbdp9088
                  Window Tint Film Chevy (back doors) DIY
                  Handcrafted Ercolano Music Box Featuring "Luncheon of the Boating Party" by Renoir, Pierre Auguste - New YorkNew York
                  MightySkins Skin Decal Wrap Compatible with Lifeproof Sticker Protective Cover 100's of Color Options
                  MightySkins Protective Vinyl Skin Decal for YETI Tundra 110 qt Cooler Lid wrap cover sticker skins Bright Smoke
                  MightySkins Skin Decal